In [39]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import logging
import warnings
from langchain.prompts import ChatPromptTemplate
from llama_cpp import Llama
import sys

logger = logging.getLogger(__name__)

# implementation of ideas found [here](https://arxiv.org/pdf/2303.02927.pdf) in a paper called LIDA

# helper functions go here


In [9]:
# def clean_code_snippet(code_string):
#     # Extract code snippet using regex
#     cleaned_snippet = re.search(r'```(?:\w+)?\s*([\s\S]*?)\s*```', code_string)

#     if cleaned_snippet:
#         cleaned_snippet = cleaned_snippet.group(1)
#     else:
#         cleaned_snippet = code_string

#     # remove non-printable characters
#     # cleaned_snippet = re.sub(r'[\x00-\x1F]+', ' ', cleaned_snippet)

#     return cleaned_snippet

In [8]:
def clean_column_names(df):
    # create a copy of the dataframe to avoid modifying the original data
    cleaned_df = df.copy()

    # iterate over column names in the dataframe
    for col in cleaned_df.columns:
        # check if column name contains any special characters or spaces
        if re.search('[^0-9a-zA-Z_]', col):
            # replace special characters and spaces with underscores
            new_col = re.sub('[^0-9a-zA-Z_]', '_', col)
            # rename the column in the cleaned dataframe
            cleaned_df.rename(columns={col: new_col}, inplace=True)

    # return the cleaned dataframe
    return cleaned_df

In [10]:
def check_type(dtype: str, value):
        """Cast value to right type to ensure it is JSON serializable"""
        if "float" in str(dtype):
            return float(value)
        elif "int" in str(dtype):
            return int(value)
        else:
            return value

In [15]:
def get_column_properties(df: pd.DataFrame, n_samples: int = 3) -> list[dict]:
  """Get properties of each column in a pandas DataFrame"""
  properties_list = []
  for column in df.columns:
      dtype = df[column].dtype
      properties = {}
      if dtype in [int, float, complex]:
          properties["dtype"] = "number"
          properties["std"] = check_type(dtype, df[column].std())
          properties["min"] = check_type(dtype, df[column].min())
          properties["max"] = check_type(dtype, df[column].max())

      elif dtype == bool:
          properties["dtype"] = "boolean"
      elif dtype == object:
          # Check if the string column can be cast to a valid datetime
          try:
              with warnings.catch_warnings():
                  warnings.simplefilter("ignore")
                  pd.to_datetime(df[column], errors='raise')
                  properties["dtype"] = "date"
          except ValueError:
              # Check if the string column has a limited number of values
              if df[column].nunique() / len(df[column]) < 0.5:
                  properties["dtype"] = "category"
              else:
                  properties["dtype"] = "string"
      elif pd.api.types.is_categorical_dtype(df[column]):
          properties["dtype"] = "category"
      elif pd.api.types.is_datetime64_any_dtype(df[column]):
          properties["dtype"] = "date"
      else:
          properties["dtype"] = str(dtype)

      # add min max if dtype is date
      if properties["dtype"] == "date":
          try:
              properties["min"] = df[column].min()
              properties["max"] = df[column].max()
          except TypeError:
              cast_date_col = pd.to_datetime(df[column], errors='coerce')
              properties["min"] = cast_date_col.min()
              properties["max"] = cast_date_col.max()
      # Add additional properties to the output dictionary
      nunique = df[column].nunique()
      if "samples" not in properties:
          non_null_values = df[column][df[column].notnull()].unique()
          n_samples = min(n_samples, len(non_null_values))
          samples = pd.Series(non_null_values).sample(n_samples, random_state=42).tolist()
          properties["samples"] = samples
      properties["num_unique_values"] = nunique
      properties["semantic_type"] = ""
      properties["description"] = ""
      properties_list.append({"column": column, "properties": properties})

  return properties_list

In [7]:
def read_dataframe(file_location):
    file_extension = file_location.split('.')[-1]
    if file_extension == 'json':
        try:
            df = pd.read_json(file_location, orient='records')
        except ValueError:
            df = pd.read_json(file_location, orient='table')
    elif file_extension == 'csv':
        df = pd.read_csv(file_location)
    elif file_extension in ['xls', 'xlsx']:
        df = pd.read_excel(file_location)
    elif file_extension == 'parquet':
        df = pd.read_parquet(file_location)
    elif file_extension == 'feather':
        df = pd.read_feather(file_location)
    elif file_extension == "tsv":
        df = pd.read_csv(file_location, sep="\t")
    elif file_extension == "txt":
        df = pd.read_csv(file_location)
    else:
        raise ValueError('Unsupported file type')

    # clean column names and check if they have changed
    cleaned_df = clean_column_names(df)
    if cleaned_df.columns.tolist() != df.columns.tolist() or len(df) > 4500:
        if len(df) > 4500:
            logger.info(f"Dataframe has more than 4500 rows. We will sample 4500 rows.")
            cleaned_df = cleaned_df.sample(4500)
        # write the cleaned DataFrame to the original file on disk
        if file_extension == 'csv':
            cleaned_df.to_csv(file_location, index=False)
        elif file_extension == 'txt':
            cleaned_df.to_csv(file_location, index=False)
        elif file_extension in ['xls', 'xlsx']:
            cleaned_df.to_excel(file_location, index=False)
        elif file_extension == 'parquet':
            cleaned_df.to_parquet(file_location, index=False)
        elif file_extension == 'feather':
            cleaned_df.to_feather(file_location, index=False)
        elif file_extension == 'json':
            with open(file_location, 'w') as f:
                f.write(cleaned_df.to_json(orient='records'))
        else:
            raise ValueError('Unsupported file type')

    return cleaned_df

# generating data summary

In [12]:
filename = "https://raw.githubusercontent.com/manavpatel1092/EDA-of-Telecom-Churn-rate/master/churn1.txt"
data = read_dataframe(filename)

In [14]:
data.head()

,State,Account_Length,Area_Code,Phone,Int_l_Plan,VMail_Plan,VMail_Message,Day_Mins,Day_Calls,Day_Charge,...,Eve_Calls,Eve_Charge,Night_Mins,Night_Calls,Night_Charge,Intl_Mins,Intl_Calls,Intl_Charge,CustServ_Calls,Churn_
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.


In [18]:
data_properties = get_column_properties(data, 3)

In [19]:
data_properties

[{'column': 'State',
  'properties': {'dtype': 'category',
   'samples': ['DC', 'WA', 'MS'],
   'num_unique_values': 51,
   'semantic_type': '',
   'description': ''}},
 {'column': 'Account_Length',
  'properties': {'dtype': 'number',
   'std': 39,
   'min': 1,
   'max': 243,
   'samples': [172, 189, 44],
   'num_unique_values': 212,
   'semantic_type': '',
   'description': ''}},
 {'column': 'Area_Code',
  'properties': {'dtype': 'number',
   'std': 42,
   'min': 408,
   'max': 510,
   'samples': [415, 408, 510],
   'num_unique_values': 3,
   'semantic_type': '',
   'description': ''}},
 {'column': 'Phone',
  'properties': {'dtype': 'string',
   'samples': ['352-6573', '369-4377', '392-2555'],
   'num_unique_values': 3333,
   'semantic_type': '',
   'description': ''}},
 {'column': 'Int_l_Plan',
  'properties': {'dtype': 'category',
   'samples': ['yes', 'no'],
   'num_unique_values': 2,
   'semantic_type': '',
   'description': ''}},
 {'column': 'VMail_Plan',
  'properties': {'dtype'

In [50]:
model_path = ".models/mistral-7b-instruct-v0.1.Q6_K.gguf"
llm = Llama(model_path=model_path, n_ctx=8192, n_batch=512,  n_threads=10, n_gpu_layers=4, verbose=False, seed=42, stream=False, chat_format="llama-2")

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from .models/mistral-7b-instruct-v0.1.Q6_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llam

In [53]:
system_prompt = """
You are an experienced data analyst that can annotate datasets. Your instructions are as follows:
1. ALWAYS generate the name of the dataset and the dataset_description
2. ALWAYS generate a field description.
3. ALWAYS generate a semantic_type (a single word) for each field given its values e.g. company, city, number, supplier, location, gender, longitude, latitude, url, ip address, zip code, email, etc
You must return an updated JSON dictionary without any preamble or explanation.
"""

base_summary = {
    "file_name": filename,
    "fields": data_properties
}

# I had to slightly change the prompt to make it work. I myself did not understand what was to be done initially.
human_prompt = f"Annotate the dictionary below. Only return a JSON object. In other words: populate the field 'description' in a meaningful way. \n {base_summary}"
system_template = {"role":"system", "content":system_prompt}
user_template = {"role":"user", "content":human_prompt}

messages = [system_template, user_template]

In [54]:
# create the generator, otherwise formatting is problematic
output = llm.create_chat_completion(messages=messages, temperature=0.0, stream=True)

In [55]:
with open("output.json", "w") as f:
  for out in output:
    try:
      pred = out['choices'][0]['delta']['content']
      f.write(pred) 
      sys.stdout.write(pred)
    except KeyError:
      pass

 {
    "file_name": "https://raw.githubusercontent.com/manavpatel1092/EDA-of-Telecom-Churn-rate/master/churn1.txt",
    "dataset_description": "Telecom Churn Dataset",
    "fields": [
        {
            "column": "State",
            "properties": {
                "dtype": "category",
                "samples": ["DC", "WA", "MS"],
                "num_unique_values": 51,
                "semantic_type": "",
                "description": "The state of the customer's location"
            }
        },
        {
            "column": "Account_Length",
            "properties": {
                "dtype": "number",
                "std": 39,
                "min": 1,
                "max": 243,
                "samples": [172, 189, 44],
                "num_unique_values": 212,
                "semantic_type": "",
                "description": "The length of the customer's account in months"
            }
        },
        {
            "column": "Area_Code",
            "properties"

' {\n    "file_name": "https://raw.githubusercontent.com/manavpatel1092/EDA-of-Telecom-Churn-rate/master/churn1.txt",\n    "fields": [\n        {\n            "column": "State",\n            "properties": {\n                "dtype": "category",\n                "samples": ["DC", "WA", "MS"],\n                "num_unique_values": 51,\n                "semantic_type": "",\n                "description": ""\n            }\n        },\n        {\n            "column": "Account_Length",\n            "properties": {\n                "dtype": "number",\n                "std": 39,\n                "min": 1,\n                "max": 243,\n                "samples": [172, 189, 44],\n                "num_unique_values": 212,\n                "semantic_type": "",\n                "description": ""\n            }\n        },\n        {\n            "column": "Area_Code",\n            "properties": {\n                "dtype": "number",\n                "std": 42,\n                "min": 408,\n       

# GOALS

# Viz

# viz explainer